In [1]:
# For sending GET requests from the API
import requests
# For saving access tokens and for file management when creating and adding to the dataset
import os
# For dealing with json responses we receive from the API
import json
# For displaying the data after
import pandas as pd
# For saving the response data in CSV format
import csv
# For parsing the dates received from twitter in readable formats
import datetime
import dateutil.parser
import unicodedata
#To add wait time between requests
import time

In [2]:
def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers

In [3]:
def create_url(keyword, max_results = 100):
    
    search_url = "https://api.twitter.com/2/tweets/search/recent" #Change to the endpoint you want to collect data from

    query_params = {'query': keyword,
                    'max_results': max_results,
                    'expansions': 'author_id,geo.place_id',
                    'tweet.fields': 'author_id,created_at,geo,id,public_metrics,text,entities',
                    'user.fields': 'id,location',
                    'place.fields': 'full_name,id,country,country_code,geo,name,place_type',
                    'next_token': {}}
    return (search_url, query_params)

In [4]:
def connect_to_endpoint(url, headers, params, next_token = None):
    params['next_token'] = next_token   #params object received from create_url function
    response = requests.request("GET", url, headers = headers, params = params)
    print("Endpoint Response Code: " + str(response.status_code))
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

In [5]:
#Inputs for the request
bearer_token = "AAAAAAAAAAAAAAAAAAAAAL1CVgEAAAAALQLs8LPJXu2qbtDpVNY8T%2BUZp30%3DrnadD6NJADffq4zIIdECRUHZk7unRPPShL8cigoKKUR9nkjsJQ"
# bearer_token = "Apna use karo"
headers = create_headers(bearer_token)
keyword = "covid lang:en"
max_results = 100

url = create_url(keyword, max_results)

In [46]:
def extract_tweet(json_response):
    data = []
    for tweet in json_response['data']:
        annotations = None
        list_of_annotations = []
        try:
            annotations = tweet['entities']['annonations']
            for annotation in annotations:
                if (annotation['type'] == 'Place'):
                    list_of_annotations.append(annotation['normalized_text'])
        except KeyError:
            pass
        obj = {
            "id": tweet['id'],
            "text": tweet['text'],
            "created_at": tweet['created_at'],
            "retweet_count": tweet['public_metrics']['retweet_count'],
            "author_id": tweet['author_id'],
            "place_id": '',
            "annotations": list_of_annotations,
        }
        try:
            obj['place_id'] = tweet['geo']['place_id']
        except KeyError:
            pass
        data.append(obj)
    
    df_data = pd.DataFrame(data)    
    df_data.to_csv('./data-9/data.csv', mode='a', index=False, header=False)

In [47]:
def extract_place(json_response):
    data=[]
    try:
        places = json_response['includes']['places']
    except KeyError:
        return
    for place in places:
        try:
            obj = {
                "id": place['id'],
                "full_name": place['full_name'],
#                 "name": place['name'],
#                 "place_type": place['place_type'],
#                 "country": place['country']
                "place": str(place)
            }
            data.append(obj)
        except KeyError:
            pass
    
    df_place = pd.DataFrame(data)
    df_place.to_csv('./data-9/place.csv', mode='a', index=False, header=False)

In [48]:
def extract_user(json_response):
    data=[]
    try:
        users = json_response['includes']['users']
    except KeyError:
        return
    for user in users:
        try:
            obj = {
                "id": user['id'],
                "location": user['location']
            }
            data.append(obj)
        except KeyError:
            pass 
     
    df_user = pd.DataFrame(data)
    df_user.to_csv('./data-9/user.csv', mode='a', index=False, header=False)

In [ ]:
# next_token = None
i = 0
for i in range(100):
    json_response = connect_to_endpoint(url[0], headers, url[1], next_token)
    next_token = json_response['meta']['next_token']
    
    extract_tweet(json_response)
    extract_user(json_response)
    extract_place(json_response)
    print(i)

Endpoint Response Code: 200
0
Endpoint Response Code: 200
1
Endpoint Response Code: 200
2
Endpoint Response Code: 200
3
Endpoint Response Code: 200
4
Endpoint Response Code: 200
5
Endpoint Response Code: 200
6
Endpoint Response Code: 200
7
Endpoint Response Code: 200
8
Endpoint Response Code: 200
9
Endpoint Response Code: 200
10
Endpoint Response Code: 200
11
Endpoint Response Code: 200
12
Endpoint Response Code: 200
13
Endpoint Response Code: 200
14
Endpoint Response Code: 200
15
Endpoint Response Code: 200
16
Endpoint Response Code: 200
17
Endpoint Response Code: 200
18
Endpoint Response Code: 200
19
Endpoint Response Code: 200
20
Endpoint Response Code: 200
21
Endpoint Response Code: 200
22
Endpoint Response Code: 200
23
Endpoint Response Code: 200
24
Endpoint Response Code: 200
25
Endpoint Response Code: 200
26
Endpoint Response Code: 200
27
Endpoint Response Code: 200
28
Endpoint Response Code: 200
29
Endpoint Response Code: 200
30
Endpoint Response Code: 200
31
Endpoint Response 